In [1]:
# importing necessary packages

import torch
import torchvision
from torchvision import models,transforms
from torch.utils.data import DataLoader
from torch import nn,optim
from collections import OrderedDict
from pathlib import Path
import pandas as pd
from PIL import Image
import matplotlib.image as mpimg
from tqdm import tqdm_notebook as tqdm
import os

In [2]:
from skll.metrics import kappa

/home/pranavsm24/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Loading the saved model 

In [3]:

state_dict = torch.load('model_checkpoint.pth')

model = models.resnet50(pretrained=True)
for params in model.parameters():
    params.requires_grad = False
    
fc  = nn.Sequential(OrderedDict([
                                ('batchnorm1',nn.BatchNorm1d(2048,eps=1e-05,momentum=0.1,affine=True)),
                                ('dropout1',nn.Dropout(p=0.5)),
                                ('fc1',nn.Linear(2048,512,bias=True)),
                                ('batchnorm2',nn.BatchNorm1d(512,eps=1e-05,momentum=0.1,affine=True)),
                                ('relu',nn.ReLU()),
                                ('dropout2',nn.Dropout(p=0.6)),
                                ('fc2',nn.Linear(512,5,bias=True)),
                                ('output',nn.LogSoftmax(dim=1))
                                ]))

AdaptiveConcatPool2d = nn.Sequential(OrderedDict([
                                                 ('ap',nn.AdaptiveAvgPool2d(output_size=(1,1))),
                                                 ('mp',nn.AdaptiveMaxPool2d(output_size=(1,1)))]))       

model.avgpool = AdaptiveConcatPool2d
model.fc = fc

In [4]:
model.load_state_dict(state_dict['state_dict'])

## Loading the images in test set

In [6]:
## Loading the DataLoader

%load_ext autoreload
%autoreload 2
from data_loader import DiabeticRetinopathyDataset

In [7]:
test_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
        
# Create the test loader

test_dataset = DiabeticRetinopathyDataset(csv_file = 'projects/Test/retinopathy_solution.csv',
                                              root_dir = 'projects/Test/Processed_test_images',
                                              transform = test_transforms)

In [8]:

test_loader = DataLoader(test_dataset,batch_size=128,shuffle=False)

## Evaluating the model

In [9]:
model = model.cuda()

In [10]:
all_true_labels = []
all_predictions = []
    
for images,labels in tqdm(test_loader):
    
    images,labels = images.cuda(),labels.cuda()
        
    model.eval()
    
    with torch.no_grad():
        output = model.forward(images)
        
        
    values = torch.exp(output)
    val_equal = values.max(dim=1)[1]
        
    all_true_labels.extend(list(labels.cpu().numpy()))
    all_predictions.extend(list(val_equal.cpu().numpy()))
        
   

In [11]:
kappa_score = kappa(all_true_labels,all_predictions,weights='quadratic')
print('The final kappa score on the test set is = {:.3f}'.format(kappa_score))

The final kappa score on the test set is = 0.360


In [23]:
print('Cl','Act','Pre')
for i in range(5):
    print(i,all_true_labels.count(i),all_predictions.count(i))


Cl Act Pre
0 39533 21307
1 3762 15259
2 7861 9652
3 1214 4284
4 1206 3074
